In [0]:
from pyspark.sql import functions as F

# 📥 Leitura da tabela Bronze (camada SOR criada anteriormente)
df_bronze = spark.table("workspace.nyc_taxi.yellowtaxi_trips_sor")

# 🎯 Seleção e filtragem para criar a camada Silver
df_silver = (
    df_bronze
    .select(
        "vendorid",              # ID do fornecedor/táxi
        "passenger_count",       # Número de passageiros
        "total_amount",          # Valor total da corrida
        "tpep_pickup_datetime",  # Data/hora de embarque
        "tpep_dropoff_datetime", # Data/hora de desembarque
        "anomes"                 # Coluna de partição no formato YYYYMM
    )
    .filter(
        (F.col("anomes") >= "202301") &  # Inclui apenas dados de janeiro/2023
        (F.col("anomes") <= "202305")    # até maio/2023 (inclusive)
    )
)

# 💾 Escrita da camada Silver
(df_silver.write
    .format("delta")
    .mode("overwrite")           # Sobrescreve a tabela se ela já existir
    .partitionBy("anomes")       # Cria partições físicas por mês
    .saveAsTable("workspace.nyc_taxi.yellowtaxi_trips_2023_spec"))

# 👀 Visualização de amostra para conferência
display(df_silver.limit(10))
